# Random Forest
### Bike Sharing Count

In [1]:
import pandas as pd

In [76]:
df = pd.read_csv("/Users/siddeg/Desktop/Bikeshare/Data/daily.csv")

In [77]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB


In [79]:
print(df.shape)

(731, 16)


### Data Preparation

#### Use Response Encoding for Categorical Variables

In [50]:
response_feat = df[['instant', 'season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']]

In [51]:
def get_gv_fea_dict(alpha, feature, df):
    value_count = df[feature].value_counts()
    gv_dict = dict()
    for i, denominator in value_count.items():
            vec = []
            for k in range(1,10):
                cls_cnt = df.loc[(df['cnt']==k) & (df[feature]==i)]
                vec.append((cls_cnt.shape[0] + alpha*10)/ (denominator + 90*alpha))
            gv_dict[i]=vec
    return gv_dict


#Get Gene variation feature
def get_gv_feature(alpha, feature, df):
  
    gv_dict = get_gv_fea_dict(alpha, feature, df)
    # value_count is similar in get_gv_fea_dict
    value_count = df[feature].value_counts()
    
    gv_fea = []
  
    for index, row in df.iterrows():
        if row[feature] in dict(value_count).keys():
            gv_fea.append(gv_dict[row[feature]])
        else:
            gv_fea.append([1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9])
#             gv_fea.append([-1,-1,-1,-1,-1,-1,-1,-1,-1])
    return gv_fea

In [52]:
alpha = 1
for col in response_feat:
    feature_responseCoding = np.array(get_gv_feature(alpha, col, df))
    response_feat[col] = feature_responseCoding

/Users/siddeg/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [54]:
response_feat.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit
0,0.10989,0.0369,0.021978,0.065789,0.0125,0.051282,0.031153,0.029674
1,0.10989,0.0369,0.021978,0.065789,0.0125,0.051282,0.031153,0.029674
2,0.10989,0.0369,0.021978,0.065789,0.0125,0.051282,0.016949,0.018083
3,0.10989,0.0369,0.021978,0.065789,0.0125,0.051546,0.016949,0.018083
4,0.10989,0.0369,0.021978,0.065789,0.0125,0.051546,0.016949,0.018083


### Reconstruct Dataset

In [118]:
semi_features = df[['temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']]

In [119]:
features = pd.concat([response_feat, semi_feature], axis=1)

In [120]:
features.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered
0,0.10989,0.0369,0.021978,0.065789,0.0125,0.051282,0.031153,0.029674,0.344167,0.363625,0.805833,0.160446,331,654
1,0.10989,0.0369,0.021978,0.065789,0.0125,0.051282,0.031153,0.029674,0.363478,0.353739,0.696087,0.248539,131,670
2,0.10989,0.0369,0.021978,0.065789,0.0125,0.051282,0.016949,0.018083,0.196364,0.189405,0.437273,0.248309,120,1229
3,0.10989,0.0369,0.021978,0.065789,0.0125,0.051546,0.016949,0.018083,0.200000,0.212122,0.590435,0.160296,108,1454
4,0.10989,0.0369,0.021978,0.065789,0.0125,0.051546,0.016949,0.018083,0.226957,0.229270,0.436957,0.186900,82,1518


### Convert Data to Arrays

In [121]:
# Use numpy to convert to arrays
import numpy as np

In [122]:
# Labels are the values we want to predict
labels = np.array(df['cnt'])

In [123]:
# Saving feature names for later use
feature_list = list(features.columns)

In [124]:
# Convert to numpy array
features = np.array(features)

### Training and Testing Sets

In [125]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

In [126]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, 
                                                                            test_size = 0.25, random_state = 42)

In [127]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (548, 14)
Training Labels Shape: (548,)
Testing Features Shape: (183, 14)
Testing Labels Shape: (183,)


### Train Model

In [128]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

In [129]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [130]:
# Train the model on training data
rf.fit(train_features, train_labels);

### Make Predictions on the Test Set

In [131]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

In [132]:
# Calculate the absolute errors
errors = abs(predictions - test_labels)

In [133]:
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 66.4 degrees.


In [134]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)

In [135]:
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 84.64 %.


### Variable Importances

In [136]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];


Variable: registered           Importance: 0.89
Variable: casual               Importance: 0.11
Variable: instant              Importance: 0.0
Variable: season               Importance: 0.0
Variable: yr                   Importance: 0.0
Variable: mnth                 Importance: 0.0
Variable: holiday              Importance: 0.0
Variable: weekday              Importance: 0.0
Variable: workingday           Importance: 0.0
Variable: weathersit           Importance: 0.0
Variable: temp                 Importance: 0.0
Variable: atemp                Importance: 0.0
Variable: hum                  Importance: 0.0
Variable: windspeed            Importance: 0.0


As we can see that most important predicting features are registered and casual. Using these 2 features